PROJECT CODE

TASKS:

1_Expansão com hidrolases - MEME and BLAST
2_Features - NS Machine learning - Clustering and Kmers
3_Functional Analysis - MOFA 
4_Estructural - Ensemble 

O que posso aplicar para a generalidade dos casos?


In [1]:
#BLAST 

from Bio.Blast import NCBIWWW, NCBIXML
import os
from Bio import SeqIO

def run_blast(query_sequence, database, output_file):
    result_handle = NCBIWWW.qblast("blastp", database, query_sequence)
    blast_records = NCBIXML.parse(result_handle)

    with open(output_file, "a") as output:
        for blast_record in blast_records:
            for alignment in blast_record.alignments:
                for hsp in alignment.hsps:
                    # Escreve os resultados no arquivo de saída
                    output.write(f"Query: {blast_record.query}\n")
                    output.write(f"Subject: {alignment.title}\n")
                    output.write(f"Score: {hsp.score}\n")
                    output.write(f"Alignment length: {hsp.align_length}\n")
                    output.write(f"E-value: {hsp.expect}\n")
                    output.write(f"Identity: {hsp.identities}\n")
                    output.write(f"Query sequence: {hsp.query}\n")
                    output.write(f"Subject sequence: {hsp.sbjct}\n\n")

# Arquivos de entrada
initial_sequences_file = "/Users/Sergiomendes/Desktop/Project/data/initial_sequences.fasta"
new_sequences_file = "/Users/Sergiomendes/Desktop/Project/data/new_sequences.fasta"
output_file = "/Users/Sergiomendes/Desktop/Project/data/blast_results.txt"

# Remover o arquivo de saída, se existir
if os.path.exists(output_file):
    os.remove(output_file)

# Executar o BLAST para cada sequência do arquivo new_sequences.fasta
with open(new_sequences_file, "r") as new_seq_file:
    for record in SeqIO.parse(new_seq_file, "fasta"):
        sequence = str(record.seq)
        # Executa o BLAST para a sequência atual
        run_blast(sequence, initial_sequences_file, output_file)


KeyboardInterrupt: 

In [24]:
#MEME

# Arquivos de entrada
initial_sequences_file = "/Users/Sergiomendes/Desktop/Project/data/initial_sequences.fasta"
new_sequences_file = "/Users/Sergiomendes/Desktop/Project/data/new_sequences.fasta"
output_file = "/Users/Sergiomendes/Desktop/Project/data/meme_results.txt"

import subprocess

# Comando para executar o programa meme e redirecionar a saída para o arquivo
command = ["/opt/local/bin/meme", initial_sequences_file, new_sequences_file, "-nmotifs", "3", "-o", output_file]

# Executar o comando
subprocess.run(command)

print("Análise MEME concluída. Os resultados foram salvos no arquivo meme_results.txt.")


Análise MEME concluída. Os resultados foram salvos no arquivo meme_results.txt.


In [ ]:

# PCA & unsupervised K-means (A adaptar)

# resfinder matriz 01
tudodegenes = []
for row in range(len(comb_file['resfinder'])):
    rf_ls = comb_file['resfinder'][row]
    tudodegenes.append(rf_ls)
flat_tudodegenes = [x for xs in tudodegenes for x in xs]
final_tg = list(dict.fromkeys(flat_tudodegenes))

# add all zeros
for i in final_tg:
    comb_file.insert(loc=9, column=i, value=0)
# if hit = 1
final_resfinder = comb_file.set_index('id')
id_rf = dict(zip(comb_file.id, final_resfinder['resfinder']))
for key, value in id_rf.items():
    for i in value:
        for z in final_resfinder.columns[8:]:
            if i == z:
                final_resfinder.loc[key, z] += 1
            else:
                continue

# standardização dos dados
scaler = StandardScaler()
final_resfinder_scaled = scaler.fit_transform(final_resfinder[final_tg])

# Pca
rf_PCA = PCA()
rf_PCA.fit(final_resfinder[final_tg])
rf_pca_reduced = rf_PCA.transform(final_resfinder[final_tg])


for x in final_resfinder["type"].unique():
    sp = comb_file.index[final_resfinder["type"] == x]
    if x == "Agricultural" or x == "Freshwater":
        plt.plot(rf_pca_reduced[sp, 0], rf_pca_reduced[sp, 1], "X", label=x, markersize=8)
    else:
        plt.plot(rf_pca_reduced[sp, 0], rf_pca_reduced[sp, 1], "o", label=x, markersize=8)

 add dashed lines for zero lines
plt.axhline(y=0, color='gray', linestyle='--', linewidth=0.5)
plt.axvline(x=0, color='gray', linestyle='--', linewidth=0.5)

plt.title("ResFinder: ARGs PCA")
plt.xlabel("P.C. 1: " + str(rf_PCA.explained_variance_ratio_[0])[0:4])
plt.ylabel("P.C. 2: " + str(rf_PCA.explained_variance_ratio_[1])[0:4])
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# set x-axis limits
plt.xlim([-6, 16])
plt.xticks(range(-5, 16, 5))

plt.show()

# k means clustering resfinder
from sklearn.cluster import KMeans

data = final_resfinder[final_tg]
pca = PCA(2)
df_1 = pca.fit_transform(data)

kmeans = KMeans(n_clusters=4, max_iter=1000)
label = kmeans.fit_predict(df_1)
centroids = kmeans.cluster_centers_
u_labels = np.unique(label)
for i in u_labels:
    plt.scatter(df_1[label == i, 0], df_1[label == i, 1], label=i)
plt.scatter(centroids[:, 0], centroids[:, 1], s=80, color='k')
plt.title("Resfinder kmeans")
plt.legend()
plt.show()

# optimal kmeans resfinder
Sum_of_squared_distances = []
K = range(1, 15)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(df_1)
    Sum_of_squared_distances.append(km.inertia_)

plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k: Resfinder')
plt.show()

final_resfinder.to_excel("C:/Users/diogo/Desktop/rf_01.xlsx")

In [ ]:
# FUNCTIONAL ANALYSIS - MOFA

import numpy as np
from mofapy2.mofapy2 import MOFA
from Bio import SeqIO
from sklearn.preprocessing import OneHotEncoder

# Carregar as sequências de proteínas do arquivo initial_sequences.fasta
initial_sequences = []
with open(inicial_sequences_file, "r") as initial_seq_file:
    for record in SeqIO.parse(initial_seq_file, "fasta"):
        initial_sequences.append(str(record.seq))

# Carregar as sequências de proteínas do arquivo new_sequences.fasta
new_sequences = []
with open(new_sequences_file, "r") as new_seq_file:
    for record in SeqIO.parse(new_seq_file, "fasta"):
        new_sequences.append(str(record.seq))

# Codificar as sequências de proteínas usando One-Hot Encoding
encoder = OneHotEncoder(dtype=np.int8)
protein_sequences = initial_sequences + new_sequences
encoded_sequences = encoder.fit_transform(protein_sequences).toarray()

# Preparar as características funcionais relevantes 
# Certificar que as características funcionais estão no formato adequado (variáveis numéricas ou categóricas)

# Criar a matriz de dados para a análise MOFA
X = np.array(encoded_sequences)

# Configurar e executar o MOFA
model = MOFA()
model.set_data_matrix(X)
model.fit(n_factors=3)  # Definir o número de fatores latentes desejado

# Analisar os resultados do MOFA
factors = model.get_factors()
loadings = model.get_loadings()

# Exemplo de impressão dos resultados
for factor_idx, factor in enumerate(factors):
    print(f"Factor {factor_idx + 1}:")
    print(factor)
    print("")

for loading_idx, loading in enumerate(loadings):
    print(f"Loading {loading_idx + 1}:")
    print(loading)
    print("")



In [ ]:
# ESTRUCTURAL ANALYSIS - ENSEMBLE

In [ ]:
# EXPANSÃO DO DATASET

import os
from Bio import SeqIO
import requests

# Define os termos de busca e filtra por proteínas da classe hidrolase
search_term = 'hidrolase NOT "PET degradation"'
search_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=protein&term={search_term}&retmax=100000'
search_result = requests.get(search_url).text
id_list = search_result.split('<Id>')[1:]
id_list = [id.split('</Id>')[0] for id in id_list]

# Baixa as sequências das proteínas encontradas
fetch_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein&id={",".join(id_list)}&rettype=fasta&retmode=text'
sequences = SeqIO.parse(requests.get(fetch_url).text, 'fasta')

# Grava as sequências em um arquivo FASTA na pasta desejada
output_folder = '/Users/Sergiomendes/Desktop/Project/Code/data'
output_file = os.path.join(output_folder, 'proteins_not_degrading_pet.fasta')
with open(output_file, 'w') as f:
    SeqIO.write(sequences, f, 'fasta')



In [ ]:
# JUNTAR OS FICHEIROS FASTA CRIANDO UM DATASET EXPANDIDO DE HIDROLASES

# Define os caminhos para os arquivos FASTA que serão combinados
file1 = '/Users/Sergiomendes/Desktop/Project/data/initial_sequences.fasta'
file2 = '/Users/Sergiomendes/Desktop/Project/data/proteins_not_degrading_pet.fasta'

# Lê as sequências de ambos os arquivos
seqs1 = SeqIO.parse(file1, 'fasta')
seqs2 = SeqIO.parse(file2, 'fasta')

# Combina as sequências em uma lista
combined_seqs = list(seqs1) + list(seqs2)

# Define o caminho para o arquivo de saída
output_folder = '/Users/Sergiomendes/Desktop/Project/data'
output_file = os.path.join(output_folder, 'expanded_dataset.fasta')

# Grava as sequências combinadas em um único arquivo FASTA
with open(output_file, 'w') as output_handle:
    SeqIO.write(combined_seqs, output_handle, 'fasta')